In [5]:
import os
import sys
import logging
import pprint
from RESTInteractions import CRABRest
from ServerUtilities import getHashLfn, encodeRequest


In [6]:
logger = logging.getLogger()
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger.setLevel(logging.DEBUG)

# if X509 vars are not defined, use default Publisher location
userProxy = os.getenv('X509_USER_PROXY')
if userProxy:
    os.environ['X509_USER_CERT'] = userProxy
    os.environ['X509_USER_KEY'] = userProxy
if not os.getenv('X509_USER_CERT'):
    os.environ['X509_USER_CERT'] = '/data/certs/servicecert.pem'
if not os.getenv('X509_USER_KEY'):
    os.environ['X509_USER_KEY'] = '/data/certs/servicekey.pem'
    
hostCert = os.getenv('X509_USER_CERT')
hostKey = os.getenv('X509_USER_KEY')

# CRAB REST API's

restHost='cmsweb-test2.cern.ch'
dbInst='dev'
crabServer = CRABRest(hostname=restHost, localcert=hostCert,
                      localkey=hostKey, retry=3,
                      userAgent='StressTest')
crabServer.setDbInstance(dbInstance=dbInst)


In [ ]:
crabS

In [3]:
def mark_good(files, crabServer, logger):
    """
    Mark the list of files as tranferred
    """

    msg = "Marking %s file(s) as published." % len(files)
    logger.info(msg)
    nMarked = 0
    for lfn in files:
        data = {}
        source_lfn = lfn
        docId = getHashLfn(source_lfn)
        data['asoworker'] = 'schedd'
        data['subresource'] = 'updatePublication'
        data['list_of_ids'] = [docId]
        data['list_of_publication_state'] = ['DONE']
        data['list_of_retry_value'] = [1]
        data['list_of_failure_reason'] = ['']

        try:
            result = crabServer.post(api='filetransfers', data=encodeRequest(data))
            logger.debug("updated DocumentId: %s lfn: %s Result %s", docId, source_lfn, result)
        except Exception as ex:
            logger.error("Error updating status for DocumentId: %s lfn: %s", docId, source_lfn)
            logger.error("Error reason: %s", ex)

        nMarked += 1
        if nMarked % 10 == 0:
            logger.info('marked %d files', nMarked)

In [5]:
# prepare a list of 10 files
lfnTemplate = '/store/temp/user/dmapelli.08aeb7b050bc579abba57f9ea54d7bc13447d65b/GenericTTbar/autotest-dmapelli-1645459633/220221_160717/0000/output_%d.root'
files=[]
for i in range(1,11):
    files.append(lfnTemplate % i)
#pprint.pprint(files)
# mark them
mark_good(files,crabServer,logger)